Importy potrzebnych modułów

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.classification import classification_report, accuracy_score, cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing.label import LabelBinarizer
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Sequential
import copy

C:\Users\pawel\anaconda3\envs\deep\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\pawel\anaconda3\envs\deep\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


Odczytanie obrazków, zamiana na rozmiar 64x64, zapis do samples i labels

In [2]:
import cv2
import os
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (64,64))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('flower_photos')
print('loaded',len(samples),' samples')
print('classes',set(labels))
org_samples = samples
org_labels = labels

loaded 3670  samples
classes {'dandelion', 'tulips', 'roses', 'daisy', 'sunflowers'}


Utworzenie modelu CNN

In [3]:
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
cnnmodel = Sequential()
cnnmodel.add(Conv2D(16, (3, 3), padding="same",input_shape=(64,64,3)))
cnnmodel.add(BatchNormalization())
cnnmodel.add(Activation("relu"))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Conv2D(32, (3, 3), padding="same"))
cnnmodel.add(Activation("relu"))
cnnmodel.add(MaxPooling2D(pool_size=(2, 2)))
cnnmodel.add(Dropout(0.25))

cnnmodel.add(Flatten())
cnnmodel.add(Dense(512))
cnnmodel.add(Activation("relu"))

cnnmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0

Zmiana typu labels

In [4]:
# one-hot labels
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print("Labels shape",labels.shape)
labels = labels.astype(float)

Labels shape (3670, 5)


Przekopiowanie samples do samples2

In [5]:
samples2 = copy.deepcopy(org_samples)
print('loaded',len(samples2),' samples')

loaded 3670  samples


Metody ekstrakcji cech dla samples2

In [6]:
import mahotas

def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

def fd_haralick(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

def fd_histogram(image, mask=None):
    bins=8
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

Stworzenie flat sampli dla modelu Dense

In [7]:
new_samples2 = []
for i,image in enumerate(samples2):
    fv_hu_moments = fd_hu_moments(image)
    fv_haralick   = fd_haralick(image)
    fv_histogram  = fd_histogram(image)
    if(i%500==0): print(i,'/',len(samples2))
    features = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
    new_samples2.append(features)
samples2 = np.array(new_samples2)
print(samples2.shape)

0 / 3670
500 / 3670
1000 / 3670
1500 / 3670
2000 / 3670
2500 / 3670
3000 / 3670
3500 / 3670
(3670, 532)


Normalizacja do przedziału 0 - 1

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
print(np.min(samples2),np.max(samples2))
samples2 = scaler.fit_transform(samples2)
print(np.min(samples2),np.max(samples2))


-0.6241640388357443 42116.99917305574
0.0 1.0000000000000002


Utworzenie modelu Dense

In [9]:
dense_model = Sequential()
dense_model.add(Dense(250, input_dim=532, activation='relu'))
dense_model.add(Dense(250, activation='relu'))

dense_model.summary()
dense_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 250)               133250    
_________________________________________________________________
dense_2 (Dense)              (None, 250)               62750     
Total params: 196,000
Trainable params: 196,000
Non-trainable params: 0
_________________________________________________________________


Konkatenacja modeli

In [10]:
combined = concatenate([cnnmodel.output, dense_model.output])
combined = Dense(16, activation="sigmoid")(combined)
combined = Dense(labels.shape[1], activation="sigmoid")(combined)

model = Model(inputs=[cnnmodel.input, dense_model.input], outputs=combined)

print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 16)   448         conv2d_input[0][0]               
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 16)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

Trenowanie nowo utworzonego modelu

In [11]:
EPOCHS = 20
BATCH=100
model.fit([samples,samples2], labels, batch_size=BATCH, epochs=EPOCHS)

Train on 3670 samples
Epoch 1/20
3670/3670 [==============================] - 11s 3ms/sample - loss: 1.5151 - accuracy: 0.3801
Epoch 2/20
3670/3670 [==============================] - 10s 3ms/sample - loss: 1.4182 - accuracy: 0.4826
Epoch 3/20
3670/3670 [==============================] - 11s 3ms/sample - loss: 1.3197 - accuracy: 0.5572
Epoch 4/20
3670/3670 [==============================] - 10s 3ms/sample - loss: 1.2265 - accuracy: 0.6019
Epoch 5/20
3670/3670 [==============================] - 10s 3ms/sample - loss: 1.1214 - accuracy: 0.6657
Epoch 6/20
3670/3670 [==============================] - 10s 3ms/sample - loss: 1.0214 - accuracy: 0.7153
Epoch 7/20
3670/3670 [==============================] - 10s 3ms/sample - loss: 0.9032 - accuracy: 0.7755
Epoch 8/20
3670/3670 [==============================] - 10s 3ms/sample - loss: 0.7914 - accuracy: 0.8297
Epoch 9/20
3670/3670 [==============================] - 10s 3ms/sample - loss: 0.6885 - accuracy: 0.8736
Epoch 10/20
3670/3670 [==========

Osiągnięte wyniki

In [12]:
results = model.predict([samples,samples2])
print(confusion_matrix(labels.argmax(axis=1), results.argmax(axis=1)))
print(classification_report(labels.argmax(axis=1), results.argmax(axis=1)))
print("Accuracy: {:.2f}".format(accuracy_score(labels.argmax(axis=1), results.argmax(axis=1))))
print("Cohen's Kappa {:.2f}".format(cohen_kappa_score(labels.argmax(axis=1), results.argmax(axis=1))))

[[633   0   0   0   0]
 [  0 896   0   2   0]
 [  0   1 638   0   2]
 [  0   0   0 699   0]
 [  0   0   0   0 799]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       633
           1       1.00      1.00      1.00       898
           2       1.00      1.00      1.00       641
           3       1.00      1.00      1.00       699
           4       1.00      1.00      1.00       799

    accuracy                           1.00      3670
   macro avg       1.00      1.00      1.00      3670
weighted avg       1.00      1.00      1.00      3670

Accuracy: 1.00
Cohen's Kappa 1.00
